In [1]:
import numpy as np
import matplotlib.pyplot as plt
import galactic_wavelets as gw
from nbodykit.lab import transform

from utils import load_boss_data

We first download and load the BOSS DR12 catalog (LOWZ South) following [nbodykit cookbook](https://nbodykit.readthedocs.io/en/latest/cookbook/boss-dr12-data.html).

In [2]:
data, randoms, cosmo = load_boss_data()

Data has already been downloaded.
Data has already been downloaded.


In [3]:
# We estimate the center of the BOSS data survey by taking averages of the ra/dec/z ranges
randra_np = randoms['RA'].compute()
randdec_np = randoms['DEC'].compute()
randz_np = randoms['Z'].compute()
randra_np[randra_np > 180] -= 360 # To make RA in [-180, 180]

In [9]:
# Determine a line of sight from the RA/DEC/z ranges
mean_ra = (randra_np.max() + randra_np.min()) / 2
mean_dec = (randdec_np.max() + randdec_np.min()) / 2
mean_z = (randz_np.max() + randz_np.min()) / 2
radecz_center = transform.SkyToCartesian(np.array([mean_ra]), np.array([mean_dec]), np.array([mean_z]), cosmo=cosmo).compute()[0]
los = radecz_center / np.linalg.norm(radecz_center) # The los goes from (0, 0, 0) to radecz_center

print("Line of sight: ", los)

Line of sight:  [0.97571154 0.01505775 0.21854119]


In [5]:
# Determine size and center of the box
rand_positions_np = randoms['Position'].compute()
BoxSize = [rand_positions_np[:, i].max() - rand_positions_np[:, i].min() for i in range(3)]
BoxSize = [BoxSize[i] * 1.05 for i in range(3)] # Add 5% to the box size to be safe for the actual data
BoxCenter = [(rand_positions_np[:, i].max() + rand_positions_np[:, i].min()) / 2 for i in range(3)]

print("Size of the box: ", BoxSize)
print("Center of the box: ", BoxCenter)

Size of the box:  [892.8683359487425, 1682.9587890141333, 945.3135243771354]
Center of the box:  [729.3254751272971, 13.789307782221158, 230.5823656426041]


In [10]:
J, Q = 4, 2
kc = 4*np.pi/3 # pixel^-1 units
kmax = 0.5 # h/Mpc units
angular_width = np.pi / 4

In [11]:
op = gw.GalaxyCatalogScatteringOp(J=J, Q=Q, kc=kc,
                                  BoxSize=BoxSize, BoxCenter=BoxCenter,
                                  los=los, los_auto_detection=False,
                                  angular_width=angular_width,
                                  scattering=True, kmax=kmax)

Auto-adjustement of the BoxSize changed BoxSize to [892.2123136195013, 1683.893662324129, 942.4777960769379] and Ngrid to (142, 268, 150).


In [11]:
wm = op(data, randoms)

Create mesh from catalog...


KeyError: 'fsky'